# Evolver Loop 1 Analysis: Feature Validation & Leakage Investigation

**Goal**: Identify and remove all post-outcome features, validate temporal legitimacy of remaining features, and establish an honest baseline.

**Critical Finding from Evaluator**: `flair_shroom` and `flair_PIF` are post-outcome rewards assigned AFTER pizza receipt - complete leakage.

In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter

# Load data
train_path = '/home/data/train.json'
with open(train_path, 'r') as f:
    train_data = json.load(f)
train_df = pd.DataFrame(train_data)

print(f"Dataset shape: {train_df.shape}")
print(f"Columns: {len(train_df.columns)}")
print("\nAll columns:")
for i, col in enumerate(train_df.columns):
    print(f"{i:2d}. {col}")

## 1. Identify Post-Outcome (Leakage) Features

Features ending with `_at_retrieval` are collected AFTER the outcome and should NOT be used.

In [ ]:
# Identify retrieval-time features (post-outcome)
retrieval_features = [col for col in train_df.columns if col.endswith('_at_retrieval')]
print(f"=== RETRIEVAL-TIME FEATURES (POST-OUTCOME - DO NOT USE) ===")
print(f"Found {len(retrieval_features)} features:")
for feature in retrieval_features:
    print(f"  - {feature}")

print(f"\n=== REQUEST-TIME FEATURES (SAFE TO USE) ===")
request_features = [col for col in train_df.columns if col.endswith('_at_request')]
print(f"Found {len(request_features)} features:")
for feature in request_features:
    print(f"  - {feature}")

print(f"\n=== OTHER FEATURES (NEED MANUAL VALIDATION) ===")
other_features = [col for col in train_df.columns 
                  if not col.endswith('_at_retrieval') 
                  and not col.endswith('_at_request')
                  and col != 'requester_received_pizza']
print(f"Found {len(other_features)} features:")
for feature in other_features:
    print(f"  - {feature}")

## 2. Investigate Flair Feature Leakage

The evaluator identified that flair features are post-outcome rewards. Let me verify this.

In [ ]:
# Analyze flair feature leakage
print("=== FLAIR FEATURE ANALYSIS ===")
print("\nFlair distribution:")
flair_counts = train_df['requester_user_flair'].value_counts(dropna=False)
print(flair_counts)

print("\n=== PIZZA RECEIPT RATE BY FLAIR ===")
flair_pizza_rates = train_df.groupby('requester_user_flair')['requester_received_pizza'].agg(['count', 'sum', 'mean'])
flair_pizza_rates.columns = ['total_users', 'users_with_pizza', 'pizza_rate']
flair_pizza_rates = flair_pizza_rates.sort_values('pizza_rate', ascending=False)
print(flair_pizza_rates)

# Check for perfect separation
print(f"\n=== PERFECT SEPARATION CHECK ===")
perfect_separation = flair_pizza_rates[flair_pizza_rates['pizza_rate'].isin([0.0, 1.0])]
print(f"Flair categories with perfect separation: {len(perfect_separation)}")
print(perfect_separation)

# Calculate total samples with perfect separation
total_perfect = perfect_separation['total_users'].sum()
print(f"\nTotal samples with perfect separation: {total_perfect} / {len(train_df)} ({total_perfect/len(train_df)*100:.1f}%)")

# Compare to overall positive rate
overall_positive_rate = train_df['requester_received_pizza'].mean()
print(f"Overall positive rate: {overall_positive_rate:.3f}")
print(f"Perfect separation samples positive rate: {perfect_separation['users_with_pizza'].sum() / total_perfect:.3f}")

print(f"\n🚨 CRITICAL: These flair categories are post-outcome rewards!")
print(f"   - 'shroom' and 'PIF' are badges given AFTER receiving pizza")
print(f"   - Using these is equivalent to using the target as a feature")
print(f"   - This explains the perfect 1.0 CV score")

## 3. Validate Remaining Features

Check each non-retrieval feature for potential leakage.

In [ ]:
# Analyze potential leakage in other features
print("=== FEATURE LEAKAGE ANALYSIS ===")

# Check features with suspiciously high correlation to target
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

# Text features (safe - available at request time)
print("\n1. TEXT FEATURES (SAFE):")
print("   - request_title, request_text, request_text_edit_aware")
print("   - These are available when the request is made")

# User activity at request time (safe)
print("\n2. USER ACTIVITY AT REQUEST TIME (SAFE):")
activity_features = [
    'requester_number_of_comments_at_request',
    'requester_number_of_posts_at_request', 
    'requester_upvotes_plus_downvotes_at_request',
    'requester_number_of_comments_in_raop_at_request',
    'requester_number_of_posts_on_raop_at_request',
    'requester_number_of_subreddits_at_request'
]
for feature in activity_features:
    if feature in train_df.columns:
        print(f"   - {feature}")

# Account age (safe)
print("\n3. ACCOUNT AGE FEATURES (SAFE):")
age_features = [
    'requester_account_age_in_days_at_request',
    'requester_days_since_first_post_on_raop_at_request'
]
for feature in age_features:
    if feature in train_df.columns:
        print(f"   - {feature}")

# Timestamp (safe)
print("\n4. TEMPORAL FEATURES (SAFE):")
print("   - unix_timestamp_of_request")
print("   - unix_timestamp_of_request_utc")

# Subreddit info (safe)
print("\n5. SUBREDDIT FEATURES (SAFE):")
print("   - requester_subreddits_at_request")

# Check post_was_edited
print("\n6. POST EDITED FLAG (NEEDS INVESTIGATION):")
print(f"   - Unique values: {train_df['post_was_edited'].unique()[:10]}")
print(f"   - Data type: {train_df['post_was_edited'].dtype}")
# Check if it's actually a boolean or a timestamp
edited_sample = train_df['post_was_edited'].iloc[0]
print(f"   - Sample value: {edited_sample}")
print(f"   - Sample type: {type(edited_sample)}")

# Check giver_username
print("\n7. GIVER USERNAME (LIKELY LEAKAGE):")
print(f"   - Unique givers: {train_df['giver_username_if_known'].nunique()}")
print(f"   - Sample values: {train_df['giver_username_if_known'].unique()[:5]}")
print(f"   - ⚠️ This is likely only known AFTER pizza is given!")

# Check request_id and username
print("\n8. IDENTIFIERS (SAFE BUT NOT PREDICTIVE):")
print("   - request_id (unique identifier)")
print("   - requester_username (may have some signal but use carefully)")

## 4. Safe Feature Set Definition

Based on analysis, define the clean feature set with NO leakage.

In [ ]:
# Define SAFE features (no leakage)
SAFE_FEATURES = {
    'text': [
        'request_title',
        'request_text', 
        'request_text_edit_aware'
    ],
    'user_activity': [
        'requester_number_of_comments_at_request',
        'requester_number_of_posts_at_request',
        'requester_upvotes_plus_downvotes_at_request',
        'requester_number_of_comments_in_raop_at_request',
        'requester_number_of_posts_on_raop_at_request',
        'requester_number_of_subreddits_at_request'
    ],
    'account_age': [
        'requester_account_age_in_days_at_request',
        'requester_days_since_first_post_on_raop_at_request'
    ],
    'temporal': [
        'unix_timestamp_of_request',
        'unix_timestamp_of_request_utc'
    ],
    'subreddit': [
        'requester_subreddits_at_request'
    ]
}

# Features to EXCLUDE (leakage)
EXCLUDE_FEATURES = [
    'requester_user_flair',  # Post-outcome reward
    'giver_username_if_known',  # Only known after pizza given
] + retrieval_features  # All _at_retrieval features

print("=== SAFE FEATURES TO USE ===")
all_safe = []
for category, features in SAFE_FEATURES.items():
    print(f"\n{category.upper()}:")
    for feature in features:
        if feature in train_df.columns:
            print(f"  ✓ {feature}")
            all_safe.append(feature)

print(f"\nTotal safe features: {len(all_safe)}")

print("\n=== FEATURES TO EXCLUDE (LEAKAGE) ===")
for feature in EXCLUDE_FEATURES:
    if feature in train_df.columns:
        print(f"  ✗ {feature}")

print(f"\nTotal excluded features: {len([f for f in EXCLUDE_FEATURES if f in train_df.columns])}")

## 5. Text Pattern Analysis

Analyze what makes a successful request in the legitimate text data.

In [ ]:
# Analyze text patterns in legitimate data (without flair leakage)
print("=== TEXT PATTERN ANALYSIS ===")

# Combine text fields
train_df['full_text'] = (train_df['request_title'].fillna('') + ' ' + 
                         train_df['request_text'].fillna('')).str.strip()

# Basic text statistics by outcome
text_stats = train_df.groupby('requester_received_pizza').agg({
    'full_text': [
        lambda x: x.str.len().mean(),
        lambda x: x.str.len().std(),
        lambda x: x.str.split().str.len().mean(),  # word count
        lambda x: x.str.split().str.len().std()
    ]
}).round(2)

text_stats.columns = ['avg_length', 'std_length', 'avg_words', 'std_words']
text_stats.index = ['No Pizza', 'Pizza']
print("\nText statistics by outcome:")
print(text_stats)

# Check for common keywords in successful vs unsuccessful requests
from collections import Counter
import re

def get_common_words(texts, n=10):
    """Get most common words from a series of texts"""
    all_text = ' '.join(texts.fillna('').astype(str).tolist())
    # Simple tokenization
    words = re.findall(r'\b[a-zA-Z]{3,}\b', all_text.lower())
    # Remove common stopwords
    stopwords = {'the', 'and', 'for', 'are', 'but', 'not', 'you', 'all', 'can', 'had', 'her', 'was', 'one', 'our', 'out', 'day', 'get', 'has', 'him', 'his', 'how', 'its', 'may', 'new', 'now', 'old', 'see', 'two', 'way', 'who', 'boy', 'did', 'she', 'use', 'her', 'now', 'old', 'see', 'two', 'way', 'who', 'oil', 'sit', 'set', 'run', 'eat', 'far', 'sea', 'eye', 'ago', 'off', 'far', 'set', 'own', 'say'}
    words = [w for w in words if w not in stopwords]
    return Counter(words).most_common(n)

print("\n=== TOP WORDS IN SUCCESSFUL REQUESTS ===")
successful_texts = train_df[train_df['requester_received_pizza'] == True]['full_text']
success_words = get_common_words(successful_texts)
for word, count in success_words:
    print(f"  {word}: {count}")

print("\n=== TOP WORDS IN UNSUCCESSFUL REQUESTS ===")
unsuccessful_texts = train_df[train_df['requester_received_pizza'] == False]['full_text']
fail_words = get_common_words(unsuccessful_texts)
for word, count in fail_words:
    print(f"  {word}: {count}")

# Analyze sentiment indicators (simple keyword-based)
sentiment_keywords = {
    'gratitude': ['thanks', 'thank', 'grateful', 'appreciate', 'bless', 'blessed'],
    'desperation': ['desperate', 'starving', 'hungry', 'broke', 'poor', 'need', 'help'],
    'storytelling': ['because', 'since', 'when', 'after', 'before', 'story'],
    'promise': ['promise', 'pay', 'forward', 'return', 'back', 'repay']
}

print("\n=== SENTIMENT KEYWORD ANALYSIS ===")
for sentiment, keywords in sentiment_keywords.items():
    print(f"\n{sentiment.upper()} keywords:")
    for keyword in keywords:
        success_count = successful_texts.str.contains(keyword, case=False, na=False).sum()
        fail_count = unsuccessful_texts.str.contains(keyword, case=False, na=False).sum()
        success_rate = success_count / len(successful_texts) if len(successful_texts) > 0 else 0
        fail_rate = fail_count / len(unsuccessful_texts) if len(unsuccessful_texts) > 0 else 0
        print(f"  {keyword:12s}: Success={success_rate:.3f}, Fail={fail_rate:.3f}")

## 6. User Activity Patterns

Analyze legitimate user activity features.

In [ ]:
# Analyze user activity patterns
print("=== USER ACTIVITY ANALYSIS ===")

activity_features = [
    'requester_number_of_comments_at_request',
    'requester_number_of_posts_at_request',
    'requester_upvotes_plus_downvotes_at_request'
]

for feature in activity_features:
    if feature in train_df.columns:
        print(f"\n{feature}:")
        stats = train_df.groupby('requester_received_pizza')[feature].agg(['mean', 'median', 'std']).round(2)
        stats.index = ['No Pizza', 'Pizza']
        print(stats)

# Account age analysis
print("\n=== ACCOUNT AGE ANALYSIS ===")
age_features = [
    'requester_account_age_in_days_at_request',
    'requester_days_since_first_post_on_raop_at_request'
]

for feature in age_features:
    if feature in train_df.columns:
        print(f"\n{feature}:")
        stats = train_df.groupby('requester_received_pizza')[feature].agg(['mean', 'median', 'std']).round(2)
        stats.index = ['No Pizza', 'Pizza']
        print(stats)

# Temporal analysis
print("\n=== TEMPORAL ANALYSIS ===")
if 'unix_timestamp_of_request' in train_df.columns:
    train_df['request_datetime'] = pd.to_datetime(train_df['unix_timestamp_of_request'], unit='s')
    train_df['request_hour'] = train_df['request_datetime'].dt.hour
    train_df['request_dayofweek'] = train_df['request_datetime'].dt.dayofweek
    
    print("\nHour of day distribution:")
    hour_stats = train_df.groupby('request_hour')['requester_received_pizza'].agg(['count', 'mean']).round(3)
    print(hour_stats.head(10))
    
    print("\nDay of week distribution:")
    day_stats = train_df.groupby('request_dayofweek')['requester_received_pizza'].agg(['count', 'mean']).round(3)
    print(day_stats)

## Summary of Findings

**SAFE FEATURES (No Leakage):**
- Text: request_title, request_text, request_text_edit_aware
- User activity: comments/posts/upvotes at request time
- Account age: days since account creation, days since first RAOP post
- Temporal: timestamp, hour, day of week
- Subreddit activity: subreddits user posts in

**LEAKAGE FEATURES (Must Exclude):**
- requester_user_flair (post-outcome reward)
- giver_username_if_known (only known after pizza given)
- All _at_retrieval features (collected after outcome)

**Key Insights:**
- Successful requests tend to be longer (more words)
- Gratitude keywords ('thanks', 'appreciate') more common in successful requests
- Storytelling elements may be important
- Account age and activity levels show some differences
- Temporal patterns may exist (hour/day of week)